In [102]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
from IPython.display import display, Markdown
import tiktoken
import os

load_dotenv(find_dotenv())

True

In [103]:
import os
from anthropic import Anthropic

client = Anthropic(
    # This is the default and can be omitted
    api_key=os.environ.get("ANTHROPIC_API_KEY"),
)


[TextBlock(text="Hello! It's nice to meet you. How can I assist you today?", type='text')]


In [129]:
OpenAI_client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
Anthropic_client = Anthropic(api_key=os.environ.get("ANTHROPIC_API_KEY"),
)

In [130]:
def get_n_tokens(text):
    enc = tiktoken.encoding_for_model("gpt-4o")
    return len(enc.encode(text))

def get_openai_answer(instructions, prompt):
    completion = OpenAI_client.chat.completions.create(
      model="gpt-4o",
      messages=[
        {"role": "system", "content": instructions},
        {"role": "user", "content": prompt}
      ]
    )
    return completion.choices[0].message.content

def get_claude_answer(instructions, prompt):
    message = client.messages.create(
        max_tokens=1024,
        system = instructions,
        messages=[
            {"role": "user", "content": prompt}
        ],
        model="claude-3-5-sonnet-20240620",
        )
    return message.content[0].text

def read_file(file_path):
    try:
        with open(file_path, 'r') as file:
            content = file.read()
        return content
    except FileNotFoundError:
        return "File not found."

def read_all_files_in_folder(folder_path):
    contents_list = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path):
            with open(file_path, 'r') as file:
                contents = file.read()
                contents_list.append(contents)
    return contents_list

def get_cost_dollars(instructions, answer):
    input_1k_tokens = 0.005
    output_1k_tokens = 0.015
    return input_1k_tokens * get_n_tokens(instructions)/1000 + output_1k_tokens * get_n_tokens(answer)/1000

def render_markdown(text):
    display(Markdown(text))

In [131]:

file_path = 'instructions.txt'
instructions = read_file(file_path)

folder_path = 'hook_examples'
all_contents = read_all_files_in_folder(folder_path)

In [132]:
final_instructions = instructions
for idx, content in enumerate(all_contents):
     final_instructions += f"""----------
                     HOOK EXAMPLE {idx}:
                     {content} \n\n\n\n\n"""

In [133]:
prompt = "I want a hook that sends an erc20 to the swapper (the hook will have a balance)"

#answer = gpt_get_answer(final_instructions, prompt)

In [136]:
final_instructions

'You are an expert solidity developer.\n\nYour task is to assist in creating hook smart contracts for Uniswap V4. \n\nUniswap v4 Hooks -- also known simply as hooks -- are specially designed contracts that run at distinct points throughout a pool action\'s lifecycle. They serve as plugins allowing developers to tailor how pools, swaps, fees, and LP positions interact. This enables innovation atop Uniswap v4\'s core features, thereby supporting the development of custom AMM pools.\n\nDuring the course of a pool action\'s lifecycle, a hook invokes custom logic primarily at four critical phases:\n\nInitialize: Activated when the pool is deployed.\nModify Position: Used to add or remove liquidity.\nSwap: Engages a swap between tokens within the V4 ecosystem.\nDonate: Facilitates the donation of liquidity to a V4 pool. Upon initialization, a pool can be associated with a hook contract. Such a contract has the ability to execute any of the callback functions during the pool action\'s lifecyc

In [134]:
answer = get_claude_answer(final_instructions, prompt)

In [126]:
get_cost_dollars(final_instructions, answer)

0.048205

In [135]:
render_markdown(answer)

Certainly! I'll create a hook that sends an ERC20 token to the swapper after a successful swap. This hook will maintain a balance of the ERC20 token and distribute it to users who perform swaps. Here's an implementation of such a hook:

```solidity
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.20;

import {BaseHook} from "v4-periphery/BaseHook.sol";
import {Hooks} from "@uniswap/v4-core/contracts/libraries/Hooks.sol";
import {IPoolManager} from "@uniswap/v4-core/contracts/interfaces/IPoolManager.sol";
import {PoolKey} from "@uniswap/v4-core/contracts/types/PoolKey.sol";
import {BalanceDelta} from "@uniswap/v4-core/contracts/types/BalanceDelta.sol";
import {IERC20} from "@openzeppelin/contracts/token/ERC20/IERC20.sol";
import {SafeERC20} from "@openzeppelin/contracts/token/ERC20/utils/SafeERC20.sol";

contract RewardHook is BaseHook {
    using SafeERC20 for IERC20;

    IERC20 public immutable rewardToken;
    uint256 public constant REWARD_AMOUNT = 1e18; // 1 token

    event RewardSent(address indexed user, uint256 amount);

    constructor(IPoolManager _poolManager, IERC20 _rewardToken) BaseHook(_poolManager) {
        rewardToken = _rewardToken;
    }

    function getHookPermissions() public pure override returns (Hooks.Permissions memory) {
        return Hooks.Permissions({
            beforeInitialize: false,
            afterInitialize: false,
            beforeAddLiquidity: false,
            afterAddLiquidity: false,
            beforeRemoveLiquidity: false,
            afterRemoveLiquidity: false,
            beforeSwap: false,
            afterSwap: true,
            beforeDonate: false,
            afterDonate: false
        });
    }

    function afterSwap(
        address sender,
        PoolKey calldata,
        IPoolManager.SwapParams calldata,
        BalanceDelta,
        bytes calldata
    ) external override returns (bytes4) {
        // Ensure the hook has enough balance to send the reward
        require(rewardToken.balanceOf(address(this)) >= REWARD_AMOUNT, "Insufficient reward balance");

        // Send the reward to the swapper
        rewardToken.safeTransfer(sender, REWARD_AMOUNT);

        emit RewardSent(sender, REWARD_AMOUNT);

        return BaseHook.afterSwap.selector;
    }

    // Function to allow the contract owner to withdraw any excess tokens
    function withdrawExcessTokens(uint256 amount) external {
        require(msg.sender == owner(), "Only owner can withdraw");
        require(rewardToken.balanceOf(address(this)) >= amount, "Insufficient balance");

        rewardToken.safeTransfer(msg.sender, amount);
    }

    // Function to check the current reward token balance of the contract
    function getRewardBalance() external view returns (uint256) {
        return rewardToken.balanceOf(address(this));
    }
}
```

This `RewardHook` contract does the following:

1. It imports necessary contracts and libraries, including `SafeERC20` for safe token transfers.

2. The contract has a state variable `rewardToken` to store the address of the ERC20 token used for rewards.

3. It defines a constant `REWARD_AMOUNT` set to 1 token (assuming 18 decimal places).

4. The constructor takes the `IPoolManager` address and the reward token address as parameters.

5. The `getHookPermissions()` function is implemented to enable only the `afterSwap` hook.

6. The `afterSwap()` function is where the main logic happens:
   - It checks if the contract has enough balance to send the reward.
   - If so, it sends the `REWARD_